# TP 5: Optimisation des Trajets des Bus Universitaires avec A*

## Hany El Atlassi - CSCC 

In [1]:
import pandas as pd

*Q 1:* Charger le CSV

In [29]:
df = pd.read_csv("data/bus_network.csv")
df.head()

,from_stop,to_stop,time_min,h_from,h_to
0,RES,P1,5,18,15
1,RES,RU,7,18,14
2,P1,SCI,6,15,9
3,P1,BIB,4,15,8
4,RU,BIB,3,14,8


In [5]:
df

,from_stop,to_stop,time_min,h_from,h_to
0,RES,P1,5,18,15
1,RES,RU,7,18,14
2,P1,SCI,6,15,9
3,P1,BIB,4,15,8
4,RU,BIB,3,14,8
5,RU,SPT,6,14,12
6,BIB,SCI,4,8,9
7,BIB,ADM,5,8,6
8,SCI,INFO,7,9,0
9,SCI,ADM,4,9,6


P1
RU


*Q 2:* Construire une structure Python sous forme de dictionnaire  
*Q 3:* Stocker séparément les heuristiqes dans un dictionnaire

In [36]:
for row in df.itertuples():
    print(row)

Pandas(Index=0, from_stop='RES', to_stop='P1', time_min=5, h_from=18, _5=15)
Pandas(Index=1, from_stop='RES', to_stop='RU', time_min=7, h_from=18, _5=14)
Pandas(Index=2, from_stop='P1', to_stop='SCI', time_min=6, h_from=15, _5=9)
Pandas(Index=3, from_stop='P1', to_stop='BIB', time_min=4, h_from=15, _5=8)
Pandas(Index=4, from_stop='RU', to_stop='BIB', time_min=3, h_from=14, _5=8)
Pandas(Index=5, from_stop='RU', to_stop='SPT', time_min=6, h_from=14, _5=12)
Pandas(Index=6, from_stop='BIB', to_stop='SCI', time_min=4, h_from=8, _5=9)
Pandas(Index=7, from_stop='BIB', to_stop='ADM', time_min=5, h_from=8, _5=6)
Pandas(Index=8, from_stop='SCI', to_stop='INFO', time_min=7, h_from=9, _5=0)
Pandas(Index=9, from_stop='SCI', to_stop='ADM', time_min=4, h_from=9, _5=6)
Pandas(Index=10, from_stop='ADM', to_stop='INFO', time_min=5, h_from=6, _5=0)
Pandas(Index=11, from_stop='SPT', to_stop='ADM', time_min=8, h_from=12, _5=6)
Pandas(Index=12, from_stop='P2', to_stop='SPT', time_min=5, h_from=16, _5=12)
Pa

In [34]:
graph = {}
heuristic = {}

for row in df.itertuples():
    if row.from_stop not in graph:
        graph[row.from_stop] = {row.to_stop: row.time_min}
    else:
        graph[row.from_stop][row.to_stop] = row.time_min
    
    if row.from_stop not in heuristic:
        heuristic[row.from_stop] = row.h_from
    
    if row.to_stop not in heuristic:
        heuristic[row.to_stop] = row._5


print(f"Graph: \n{graph}\n\nHeuristic: \n{heuristic}")

Graph: 
{'RES': {'P1': 5, 'RU': 7}, 'P1': {'SCI': 6, 'BIB': 4}, 'RU': {'BIB': 3, 'SPT': 6}, 'BIB': {'SCI': 4, 'ADM': 5}, 'SCI': {'INFO': 7, 'ADM': 4}, 'ADM': {'INFO': 5}, 'SPT': {'ADM': 8}, 'P2': {'SPT': 5, 'P1': 6}}

Heuristic: 
{'RES': 18, 'P1': 15, 'RU': 14, 'SCI': 9, 'BIB': 8, 'SPT': 12, 'ADM': 6, 'INFO': 0, 'P2': 16}


*Q 4:* Implémentation de l'algorithme A* pour le réseau de bus

In [38]:
import heapq

def a_star_search_bus(graph, heuristic, start, goal):
    if start not in graph:
        print(f"Starting stop '{start}' not found in the graph")
        return None, None
    if goal not in heuristic:
        print(f"Goal stop '{goal}' not found in heuristic")
        return None, None
    
    if start == goal:
        print("Already at the destination")
        return [start], 0
    
    open_list = []
    heapq.heappush(open_list, (heuristic.get(start, 0), 0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic.get(start, 0)}
    closed_set = set()
    
    while open_list:
        current_f, current_g, current = heapq.heappop(open_list)
        
        if current == goal:
            path = []
            total_cost = g_score[current]
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            print(f"Path found: {' -> '.join(path)}")
            print(f"Total time: {total_cost} minutes")
            return path, total_cost
        
        closed_set.add(current)
        
        if current in graph:
            for neighbor, edge_cost in graph[current].items():
                if neighbor in closed_set:
                    continue
                
                tentative_g = g_score[current] + edge_cost
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    h = heuristic.get(neighbor, 0)
                    f_score[neighbor] = tentative_g + h
                    heapq.heappush(open_list, (f_score[neighbor], tentative_g, neighbor))
    
    print(f"No path found from {start} to {goal}")
    return None, None

# Test
start_stop = "RES"
goal_stop = "INFO"
path, cost = a_star_search_bus(graph, heuristic, start_stop, goal_stop)


Path found: RES -> P1 -> SCI -> INFO
Total time: 18 minutes


*Q 5:* Implémentation de Greedy Best First 

In [39]:
import heapq

def greedy_best_first_search(graph, heuristic, start, goal):
    if start not in graph:
        print(f"Starting stop '{start}' not found in the graph")
        return None, None
    if goal not in heuristic:
        print(f"Goal stop '{goal}' not found in heuristic")
        return None, None
    
    if start == goal:
        print("Already at the destination")
        return [start], 0
    
    open_list = []
    heapq.heappush(open_list, (heuristic.get(start, 0), start))
    
    came_from = {}
    g_score = {start: 0}  
    closed_set = set()
    
    while open_list:
        current_h, current = heapq.heappop(open_list)
        
        if current == goal:
            path = []
            total_cost = g_score[current]
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            print(f"Path found: {' -> '.join(path)}")
            print(f"Total time: {total_cost} minutes")
            return path, total_cost
        
        closed_set.add(current)
        
        if current in graph:
            for neighbor, edge_cost in graph[current].items():
                if neighbor in closed_set:
                    continue
                
                tentative_g = g_score[current] + edge_cost
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    
                    h = heuristic.get(neighbor, 0)
                    heapq.heappush(open_list, (h, neighbor))
    
    print(f"No path found from {start} to {goal}")
    return None, None


path_greedy, cost_greedy = greedy_best_first_search(graph, heuristic, "RES", "INFO")



Path found: RES -> RU -> BIB -> ADM -> INFO
Total time: 20 minutes
